<a href="https://colab.research.google.com/github/donguk99/TSP/blob/master/mTSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**mTSP(m=1 -> TSP)**

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

file = open('/content/ftv33.atsp', 'r')

# Read data (.atsp) 
Name = file.readline().strip().split()[1] # NAME
FileType = file.readline().strip().split()[1] # TYPE
Comment = file.readline().strip() # COMMENT
Dimension = file.readline().strip().split()[1] # DIMENSION
EdgeWeightType = file.readline().strip().split()[1] # EDGE_WEIGHT_TYPE
EdgeWeightFormat = file.readline().strip().split()[1]
file.readline()
n=int(Dimension)
d=np.zeros((n,n), int)

# graph
row = []
next_row=[]
i=0
while i<n:
  if len(row)==0:
    row = next_row[:]
    next_row=[]
    row += list(map(int, file.readline().strip().split()))
  elif len(row)<n:
    row += list(map(int, file.readline().strip().split()))
  elif len(row)>n:
      next_row = row[n:]
      row = row[:n]
  else:
    d[i][:] = np.array(row, int)
    row=[]
    i+=1

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
# mTSP data example
d=[[0,      4.3092, 3.8329, 2.7762, 1.7441, 4.2076, 6.0199],
   [4.3092, 0,      1.6596, 3.7435, 5.6408, 2.5764, 7.5691],
   [3.8329, 1.6596, 0,      2.3055, 4.7278, 3.8966, 8.4056],
   [2.7762, 3.7435, 2.3055, 0,      2.8311, 5.2142, 8.5694],
   [1.7441, 5.6408, 4.7278, 2.8311, 0,      5.9246, 7.3483],
   [4.2076, 2.5764, 3.8966, 5.2142, 5.9246, 0,      5.1733],
   [6.0199, 7.5691, 8.4056, 8.5694, 7.3483, 5.1733, 0    ]]
d=np.array(d)
n=7
m=3 # number of salesman


multi depot인 경우 실행

In [ ]:
# 0번째 도시를 더미도시(모든 도시로의 거리 0)로 지정하여 각각 다른 곳에서 출발하도록 설정
d = np.pad(d, pad_width=((1,0),(1,0)), mode='constant', constant_values=0)
n = n+1

초기 object, constraints 추가

In [ ]:
# objective, constraints
m=2
x=cp.Variable((n, n), boolean=True)
objective = cp.Minimize(cp.trace(d.T@x))
ones= np.ones((n,1))
constraints = []
constraints += [x[0,:]@ones == m] # m명이 출발노드에서 출발  # m==1이면 TSP problem
constraints += [x[:,0]@ones == m] # m명이 출발노드로 도착
constraints += [x[1:,:]@ones == 1] # 각 노드는 1명에 의해 한번만 방문 
constraints += [x[:,1:].T@ones == 1]
constraints += [cp.diag(x) == 0]

subtours를 찾아주는 함수

In [ ]:
def get_subtours(routes,m)->list:
  # covert routes to subtours
  # example) [(0,1), (1,2), (2,0), (3,4), (4, 3)] -> [(0,1,2), (3,4)]
  # return subtours

  subtours = []
  r=routes.copy()

  # depart
  depart = r[:m]
  r= r[m:]

  # route (salesman)
  for route in depart:
    route=list(route)
    while r and route[-1] != 0:
      for i, j in enumerate(r):
        if route[-1] == j[0]:
            route.append(j[1])
            del (r[i])
            break
    if route[-1] == 0:
      route.pop()
    subtours.append(route)

  # route (left)
  while len(r) != 0:
    route = list(r.pop())
    l = 0
    while len(route) > l:
      l = len(route)
      for i, j in enumerate(r):
        if route[-1] == j[0]:
          if j[1] != route[0]:
            route.append(j[1])
            del (r[i])
          else:
            del (r[i])
    subtours.append(route)

  return subtours

subtour의 개수가 m개가 될 때까지 구속조건을 추가하며 반복

In [ ]:
# mTSP subtour elimination
iter = 0
subtours=[]
while len(subtours)!=m:
  prob = cp.Problem(objective, constraints)
  prob.solve()

  # find subtours
  routes=[(i,j) for i in range(n) for j in range(n) if x[i,j].value==1]
  subtours = get_subtours(routes,m)

  
  print(f'iteration:{iter}')
  print(f'number of constraints:{len(constraints)}')
  print(f'number of subtours:{len(subtours)}')
  print(f'cost:{prob.value}')
  print(f'subtour:{subtours} \n')

  S=[]
  for subtour in subtours:
    V=[0 for _ in range(n)]
    for i in subtour:
      V[i] = 1
    V=np.array(V)
    S.append(V)

  # add constraints
  for V in S:
    if m==1:
      constraints += [V@(x@V.T) <= sum(V)-1]
    elif V[0] != 1:
      constraints += [V@(x@V.T) <= sum(V)-1]
  iter+=1

# print(f'iteration:{iter}')
# print(f'number of constraints:{len(constraints)}')
# print(f'number of subtours:{len(subtours)}')
# print(f'cost:{prob.value}')
# print(f'subtour:{subtours} \n')
      


iteration:0
number of constraints:5
number of subtours:3
cost:10.029699999999998
subtour:[[0, 4, 3, 2, 6], [0, 7], [5, 1]] 

iteration:1
number of constraints:6
number of subtours:4
cost:10.6706
subtour:[[0, 6], [0, 7], [5, 1, 4], [3, 2]] 

iteration:2
number of constraints:8
number of subtours:2
cost:11.061799999999998
subtour:[[0, 5, 1, 4, 3, 2, 6], [0, 7]] 



# 새 섹션

In [ ]:
from google.colab import drive
drive.mount('/content/drive')